In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [2]:
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

In [3]:
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [4]:
# Additional imports for loading PDF documents and QA chain.
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [5]:
# Additional imports for loading Wikipedia content and QA chain.
from langchain_community.document_loaders import WikipediaLoader
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# Create the PDF loader and load the document.
#pdf_loader = PyPDFLoader('https://static.bc-edx.com/ai/ail-v-1-0/m22/lesson_1/datasets/Resume.pdf')
#documents = pdf_loader.load()

# Initialize the model.
#llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the QA chain using the LLM.
#chain = load_qa_chain(llm)

# Define a query as a string.
#query = 'Could I write to Jane Doe in Spanish and expect her to understand?'

# Pass the documents and the query to the chain, and print the result.
#result = chain.invoke({"input_documents": documents, "question": query})
#print(result["output_text"])

In [6]:
#Setting the Model

# Initialize the model with your OpenAI API key
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = 'gpt-3.5-turbo'  # We're using GPT 3.5 turbo as we don't really need GPT 4 for this as the info doesn't change that much. It provides what we need at a lower cost.
#Instantiating the llm we'll use and the arguments to pass
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Define the wikipedia topic as a string.
wiki_topic = "diabetes"

# Load the wikipedia results as documents, using a max of 2.
#included error handling- unable to load documents
try:
    documents = WikipediaLoader(query=wiki_topic, load_max_docs=2, load_all_available_meta=True).load()
except Exception as e:
    print("Failed to load documents:", str(e))
    documents = []
# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

In [7]:

#Define the function to call the LLM
def handle_query(user_query):
    if not documents:
        return "Source not loading info; please try again later."

    if user_query.lower() == 'quit':
        return "Goodbye!"

    try:
        # Pass the documents and the user's query to the chain, and return the result.
        result = chain.invoke({"input_documents": documents, "question": user_query})
        if result["output_text"].strip():
            return result["output_text"]
        else:
            return "I'm sorry. I'm not finding an good answer for you. Please rephrase your question or try another question."
    except Exception as e:
        return "An error occurred while searching for the answer: " + str(e)




BELOW CODE IS SIMPLE GRADIO AP - NO HISTORY - 

In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=handle_query,  # Callback function
    inputs=gr.Textbox(lines=2, placeholder="What is your question about diabetes?"),
    outputs=gr.Textbox(),
    title="Diabetes Information Query",
    description="Enter your question about diabetes and get information from Wikipedia."
)

# Run the Gradio app
iface.launch()

BELOW CODE BLOCK  INCLUDES THE HANDLE QUERY FUNCTION AND HAS HISTORY IN APP UI

In [8]:
# query function with history

def handle_query(user_query, history):
    if not documents:
        return "Source not loading info; please try again later.", history

    if user_query.lower() == 'quit':
        return "Goodbye!", history

    try:
        result = chain.invoke({"input_documents": documents, "question": user_query})
        ai_response = result["output_text"].strip() if result["output_text"].strip() else "I'm sorry. No good answer found. Please try another question."
        updated_history = history + [(user_query, ai_response)]
        return ai_response, updated_history
    except Exception as e:
        return f"An error occurred: {str(e)}", history

with gr.Blocks() as app:
    with gr.Row():
        user_input = gr.Textbox(label="Enter your question about diabetes", lines=2, placeholder="What is your question about diabetes?")
        submit_button = gr.Button("Submit")
    output = gr.Textbox(label="AI Response")
    history_display = gr.Textbox(label="Conversation History", visible=True) # Set visible to True to show the history

    history_state = gr.State()  # Initialize state; no default value needed

    def update_interface(user_query, history):
        if history is None:
            history = []  # Ensure history starts as an empty list

        response, updated_history = handle_query(user_query, history)
        return response, updated_history, updated_history  # Update output, history display, and state

    submit_button.click(
        fn=update_interface,
        inputs=[user_input, history_state],
        outputs=[output, history_display, history_state]
    )

app.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
